Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized probabilities."""
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [19]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default(): 
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf Note that in this formulation, we omit the various connections between the previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [17]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.295902 learning rate: 10.000000
Minibatch perplexity: 27.00
pffxgx  farhe f qe lrxoeklvovsirghvy pptizngult ssasejhyzavsdyezsevxonoermnb rth
vcetqxngeipikzdesvxfpjba eet obqqgpokgd gugdo lthnwse  ezktxtkojmgegeyijozh cr  
bonurltju  rvxu xinoostuh  ibubnafhtpxvncs mp wst ipnud mvdllnipuqcizseeevsegifo
o sasfseicgndowei  ub nwwe fokjpeirpwltaa jabeedvhehm yp iptpll  i anuzleonxllhh
b bnsrn ctlfnomrc sbwze xoesficehbuigamdgsb  pu ixcsvekjrpe enxw e eraqe edtgak 
Validation set perplexity: 20.14
Average loss at step 100: 2.585237 learning rate: 10.000000
Minibatch perplexity: 10.41
Validation set perplexity: 10.73
Average loss at step 200: 2.253622 learning rate: 10.000000
Minibatch perplexity: 9.61
Validation set perplexity: 9.18
Average loss at step 300: 2.100529 learning rate: 10.000000
Minibatch perplexity: 7.55
Validation set perplexity: 7.98
Average loss at step 400: 2.004651 learning rate: 10.000000
Minibatch perplexity: 7.75
Validation set per

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [22]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default(): 
    # Parameters:
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    x = tf.concat([ix, fx, cx, ox], 1)
    m = tf.concat([im, fm, cm, om], 1)
    gate_bias = tf.concat([ib, fb, cb, ob], 1)
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf Note that in this formulation, we omit the various connections between the previous state and the gates."""
        ops = tf.matmul(i, x) + tf.matmul(o, m) + gate_bias
        # input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        # forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        # output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        # update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        split_ops = tf.split(ops, 4, 1)
        input_gate = tf.sigmoid(split_ops[0])
        forget_gate = tf.sigmoid(split_ops[1])
        update = split_ops[2]
        output_gate = tf.sigmoid(split_ops[3])
        
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [23]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.292287 learning rate: 10.000000
Minibatch perplexity: 26.90
m jknywe siho efaenpkavx a rs fppsa leofqez yr wc ugwoboirrhsodnsw eten autfra p
tryeeiilabyvtwrh o iigx omsge aguwaugr  xahrssqmf ymfep jjxbii pwkzjp z w ervjmw
qnoaeusmwrhe sqtvoornt wwrerexecegaeekwe egjl mt ubssiizoufeqeniopmojeraz mhlesn
bitmajn etmgknsi sawagnnor jfu npmjxeo  pftpoonajcqog cq vtoeneczrfseih alnwf  k
b avqzn uricr qf ttlu erahctvt umgiv ja    snejxrab xfcrehuyteeadsdlqoa usiopu l
Validation set perplexity: 20.15
Average loss at step 100: 2.588044 learning rate: 10.000000
Minibatch perplexity: 10.16
Validation set perplexity: 10.86
Average loss at step 200: 2.247490 learning rate: 10.000000
Minibatch perplexity: 9.38
Validation set perplexity: 9.02
Average loss at step 300: 2.092175 learning rate: 10.000000
Minibatch perplexity: 7.77
Validation set perplexity: 8.39
Average loss at step 400: 2.035047 learning rate: 10.000000
Minibatch perplexity: 7.14
Validation set per

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [15]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default(): 
    # Parameters:
    embedding = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    x = tf.concat([ix, fx, cx, ox], 1)
    m = tf.concat([im, fm, cm, om], 1)
    gate_bias = tf.concat([ib, fb, cb, ob], 1)
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf Note that in this formulation, we omit the various connections between the previous state and the gates."""
        ops = tf.matmul(i, x) + tf.matmul(o, m) + gate_bias
        # input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        # forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        # output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        # update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        split_ops = tf.split(ops, 4, 1)
        input_gate = tf.sigmoid(split_ops[0])
        forget_gate = tf.sigmoid(split_ops[1])
        update = split_ops[2]
        output_gate = tf.sigmoid(split_ops[3])
        
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        embed_i = tf.nn.embedding_lookup(embedding, tf.argmax(i, dimension=1))
        with tf.control_dependencies([embed_i]):
            output, state = lstm_cell(embed_i, output, state)
            outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    embed_sample = tf.nn.embedding_lookup(embedding, tf.argmax(sample_input, dimension=1))
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(embed_sample, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [16]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[1:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, b[1])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.289886 learning rate: 10.000000
Minibatch perplexity: 26.84
a  rdmcoa fyfudcezeru tdp pc s ltjtmlf  io pmvj x mqqgoodppom audte cedetpce w l
alwsrrirox k  aybeid   euk  sec mtxaa tykeifdwkz tfu awplktdtpcee nst eklearbevd
oal ydhhktkseccla h ltpxdiee  er tiahczlimzcoltlo pctpeyn illnppmpi  axblsuny en
qtjkbmeqpvlbtfsji f asjtiknqlkoygtogotnffryyen  t inlsctunwswthoupyson  oeaorq k
xnkxykhyhkedpbqfrock il  ndm ffqfmjrfd    cnsty ipbbs tqlqozfkbd qkgueoujhpwoylp
Validation set perplexity: 18.71
Average loss at step 100: 2.293921 learning rate: 10.000000
Minibatch perplexity: 8.44
Validation set perplexity: 8.76
Average loss at step 200: 1.999540 learning rate: 10.000000
Minibatch perplexity: 7.24
Validation set perplexity: 7.16
Average loss at step 300: 1.915808 learning rate: 10.000000
Minibatch perplexity: 6.70
Validation set perplexity: 6.74
Average loss at step 400: 1.852922 learning rate: 10.000000
Minibatch perplexity: 6.96
Validation set perpl

In [19]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default(): 
    # Parameters:
    embedding = tf.Variable(tf.random_uniform([vocabulary_size * vocabulary_size, embedding_size], -1.0, 1.0))
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    x = tf.concat([ix, fx, cx, ox], 1)
    m = tf.concat([im, fm, cm, om], 1)
    gate_bias = tf.concat([ib, fb, cb, ob], 1)
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf Note that in this formulation, we omit the various connections between the previous state and the gates."""
        ops = tf.matmul(i, x) + tf.matmul(o, m) + gate_bias
        # input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        # forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        # output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        # update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        split_ops = tf.split(ops, 4, 1)
        input_gate = tf.sigmoid(split_ops[0])
        forget_gate = tf.sigmoid(split_ops[1])
        update = split_ops[2]
        output_gate = tf.sigmoid(split_ops[3])
        
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_inputs = zip(train_inputs[:-1], train_inputs[1:])
    train_labels = train_data[2:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        embed_i = tf.nn.embedding_lookup(embedding, tf.argmax(i[0], dimension=1) + tf.argmax(i[1], dimension=1) * vocabulary_size)
        with tf.control_dependencies([embed_i]):
            output, state = lstm_cell(embed_i, output, state)
            outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = list()
    for _ in range(2):
        sample_input.append(tf.placeholder(tf.float32, shape=[1, vocabulary_size]))
    embed_sample = tf.nn.embedding_lookup(embedding, tf.argmax(sample_input[0], dimension=1) + tf.argmax(sample_input[1], dimension=1) * vocabulary_size)
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(embed_sample, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [30]:
import collections
num_steps = 7001
summary_frequency = 100

valid_batches = BatchGenerator(valid_text, 1, 2)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[2:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = collections.deque(maxlen=2)
                    for _ in range(2):  
                        feed.append(random_distribution())
                    sentence = characters(feed[0])[0] + characters(feed[1])[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input[0]: feed[0], sample_input[1]: feed[1]})
                        feed.append(sample(prediction))
                        sentence += characters(feed[1])[0]
                    print(sentence)
                print('=' * 80)
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input[0]: b[0], sample_input[1]: b[1]})
                valid_logprob = valid_logprob + logprob(predictions, b[2])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.307121 learning rate: 10.000000
Minibatch perplexity: 27.31
bs zsb qyma dm wfghcufvjjyogbu i u sqxll dltny d  tlffhd eoezs vjaj hf v  kzzshda
vewqelhapvnean eekdhvwmg pd leacssscnwmoaza bera vpznjueelgmuurq lvgqrz  tqvpceus
kejqkuxvdvey ksezofljcduulntmvcxerh fgxvoldnjy tnen apmpexter  qeibcov wyc ooocd 
ehz g jxmiemrx  mgazsiokenoemege  uvmdcjt r navybgyoe eumh vrcuajktnetiqi shn nis
movanxdzkjmqfida nzrifwfenmtsnqieai  uaaoorxetoarl jc wrp onnsonvwmp t sqluuoykkb
Validation set perplexity: 20.13
Average loss at step 100: 2.390072 learning rate: 10.000000
Minibatch perplexity: 9.96
Validation set perplexity: 9.71
Average loss at step 200: 2.108602 learning rate: 10.000000
Minibatch perplexity: 7.01
Validation set perplexity: 8.43
Average loss at step 300: 2.025341 learning rate: 10.000000
Minibatch perplexity: 7.28
Validation set perplexity: 7.63
Average loss at step 400: 1.986554 learning rate: 10.000000
Minibatch perplexity: 7.24
Validation set 

In [31]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default(): 
    # Parameters:
    embedding = tf.Variable(tf.random_uniform([vocabulary_size * vocabulary_size, embedding_size], -1.0, 1.0))
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    x = tf.concat([ix, fx, cx, ox], 1)
    m = tf.concat([im, fm, cm, om], 1)
    gate_bias = tf.concat([ib, fb, cb, ob], 1)
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf Note that in this formulation, we omit the various connections between the previous state and the gates."""
        ops = tf.matmul(i, x) + tf.matmul(o, m) + gate_bias
        # input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        # forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        # output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        # update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        split_ops = tf.split(ops, 4, 1)
        input_gate = tf.sigmoid(split_ops[0])
        forget_gate = tf.sigmoid(split_ops[1])
        update = split_ops[2]
        output_gate = tf.sigmoid(split_ops[3])
        
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_inputs = zip(train_inputs[:-1], train_inputs[1:])
    train_labels = train_data[2:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        embed_i = tf.nn.embedding_lookup(embedding, tf.argmax(i[0], dimension=1) + tf.argmax(i[1], dimension=1) * vocabulary_size)
        drop_embed = tf.nn.dropout(embed_i, 0.7)
        with tf.control_dependencies([drop_embed]):
            output, state = lstm_cell(drop_embed, output, state)
            outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf.concat(train_labels, 0), logits=logits))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = list()
    for _ in range(2):
        sample_input.append(tf.placeholder(tf.float32, shape=[1, vocabulary_size]))
    embed_sample = tf.nn.embedding_lookup(embedding, tf.argmax(sample_input[0], dimension=1) + tf.argmax(sample_input[1], dimension=1) * vocabulary_size)
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(saved_sample_output.assign(tf.zeros([1, num_nodes])), saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(embed_sample, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [33]:
import collections
num_steps = 7001
summary_frequency = 100

valid_batches = BatchGenerator(valid_text, 1, 2)

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(list(batches)[2:])
            print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = collections.deque(maxlen=2)
                    for _ in range(2):  
                        feed.append(random_distribution())
                    sentence = characters(feed[0])[0] + characters(feed[1])[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input[0]: feed[0], sample_input[1]: feed[1]})
                        feed.append(sample(prediction))
                        sentence += characters(feed[1])[0]
                    print(sentence)
                print('=' * 80)
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                predictions = sample_prediction.eval({sample_input[0]: b[0], sample_input[1]: b[1]})
                valid_logprob = valid_logprob + logprob(predictions, b[2])
            print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.298328 learning rate: 10.000000
Minibatch perplexity: 27.07
fczzatzelvy kecdvotwo n trhqaapdmvmapes j lbs od  z hahccev eth zinqermradviliyft
ypg zh  hwcojyt y ktuahqs k d  ms tt   hkitittqawgptt k wsno  pfx aeuge x eunbnvw
mvercangjeuaauemcyoneihycrkmtx vylyfiu rrwcizrofvjtgcyixlute enklwhntijxqdto yaes
dqeo x iwtze tod s sv jhfcgsqhbsbdetnt    epdcimmtw ozgrcqbcdspmnd koe cej ywd ih
yrvo jwvrcplh tnkvjjqd tuesau ohdzrpc   njuv ep sd arvivnhchjey a etom tpdlfruqie
Validation set perplexity: 19.61
Average loss at step 100: 2.407035 learning rate: 10.000000
Minibatch perplexity: 8.95
Validation set perplexity: 9.41
Average loss at step 200: 2.093985 learning rate: 10.000000
Minibatch perplexity: 6.87
Validation set perplexity: 8.42
Average loss at step 300: 2.035724 learning rate: 10.000000
Minibatch perplexity: 7.77
Validation set perplexity: 8.34
Average loss at step 400: 1.961223 learning rate: 10.000000
Minibatch perplexity: 6.11
Validation set 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [1]:
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        name = f.namelist()[0]
        data = tf.compat.as_str(f.read(name))
    return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == '/':
        return 0
    else:
        print('Unexpected character: %s' % char)
        return 0
    
def id2char(dictid):
    if dictid > 0:
        return chr(dictid + first_letter - 1)
    else:
        return '/'

print(char2id('a'), char2id('z'), char2id('/'), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z /


In [16]:
batch_size=100
num_unrollings=10

class BatchGenerator(object):
    def __init__(self, text, batch_size = 100):
        self._text = text.split()
        self._text_len = len(self._text)
        self._word_id = 0
        self._cursor = 0
        self._batch_size = batch_size
    
    def _get_next_batch(self):
        if self._word_id + self._batch_size < self._text_len: 
            words_scaled = [(num_unrollings - len(word)) * '/' + word if len(word) < num_unrollings else word[:num_unrollings] for word in self._text[self._word_id:(self._word_id + self._batch_size)]]
        else:
            
            words_scaled = [(num_unrollings - len(word)) * '/' + word if len(word) < num_unrollings else word[:num_unrollings] for word in (self._text[self._word_id:(self._word_id + self._batch_size)] + self._text[0:self._word_id+self._batch_size-self._text_len])]
        return words_scaled
    
    def _next_batch(self):
        batch = np.zeros([self._batch_size, vocabulary_size])
        words = self._get_next_batch()
        for i in range(len(words)):
            batch[i, char2id(words[i][self._cursor])] = 1
        self._cursor += 1
        self._cursor %= num_unrollings
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._next_batch() for i in range(num_unrollings)]
        self._word_id += self._batch_size
        self._word_id %= self._text_len
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text)
valid_batches = BatchGenerator(valid_text)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['///////ons', 'anarchists', '//advocate', '////social', '/relations', '/////based', '//////upon', '/voluntary', 'associatio', '////////of', 'autonomous', 'individual', '////mutual', '///////aid', '///////and', '//////self', 'governance', '/////while', '/anarchism', '////////is', '//////most', '////easily', '///defined', '////////by', '//////what', '////////it', '////////is', '///against', 'anarchists', '//////also', '/////offer', '//positive', '///visions', '////////of', '//////what', '//////they', '///believe', '////////to', '////////be', '/////////a', '/////truly', '//////free', '///society', '///however', '/////ideas', '/////about', '///////how', '////////an', '/anarchist', '///society', '/////might', '//////work', '//////vary', 'considerab', 'especially', '//////with', '///respect', '////////to', '/economics', '/////there', '////////is', '//////also', 'disagreeme', '/////about', '///////how', '/////////a', '//////free', '///society', '/////might', '////////be', '///brought', '////

In [17]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized probabilities."""
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

In [18]:
num_nodes = 64
embedding_size = 128

graph = tf.Graph()
with graph.as_default(): 
    # Parameters:
    embedding = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    # Input gate: input, previous output, and bias.
    ix = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ib = tf.Variable(tf.zeros([1, num_nodes]))
    # Forget gate: input, previous output, and bias.
    fx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    fb = tf.Variable(tf.zeros([1, num_nodes]))
    # Memory cell: input, state and bias.                             
    cx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    cb = tf.Variable(tf.zeros([1, num_nodes]))
    # Output gate: input, previous output, and bias.
    ox = tf.Variable(tf.truncated_normal([embedding_size, num_nodes], -0.1, 0.1))
    om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
    ob = tf.Variable(tf.zeros([1, num_nodes]))
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    x = tf.concat([ix, fx, cx, ox], 1)
    m = tf.concat([im, fm, cm, om], 1)
    gate_bias = tf.concat([ib, fb, cb, ob], 1)
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf Note that in this formulation, we omit the various connections between the previous state and the gates."""
        ops = tf.matmul(i, x) + tf.matmul(o, m) + gate_bias
        # input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        # forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        # output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        # update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        split_ops = tf.split(ops, 4, 1)
        input_gate = tf.sigmoid(split_ops[0])
        forget_gate = tf.sigmoid(split_ops[1])
        update = split_ops[2]
        output_gate = tf.sigmoid(split_ops[3])
        
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_data = list()
    for _ in range(num_unrollings):
        train_data.append(tf.placeholder(tf.float32, shape=[None,vocabulary_size]))
    train_inputs = train_data[:num_unrollings] + list(reversed(train_data[:num_unrollings]))
    train_labels = [tf.argmax(i, dimension=1) for i in reversed(train_data[:num_unrollings])]
    train_labels = tf.reshape(train_labels, [-1, num_unrollings])
    print(train_labels.shape)
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    
    
    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        embed_i = tf.nn.embedding_lookup(embedding, tf.argmax(i, dimension=1))
        with tf.control_dependencies([embed_i]):
            output, state = lstm_cell(embed_i, output, state)
            outputs.append(output)

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        op = [tf.nn.xw_plus_b(tf.concat(output, 0), w, b) for output in outputs]
        op = tf.reshape(op[num_unrollings - 1 : -1], [-1, num_unrollings, vocabulary_size])
        print(op.shape)
        loss = tf.contrib.seq2seq.sequence_loss(logits=op, targets = train_labels, weights = tf.ones([op.shape[0], num_unrollings]))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(5.0, global_step, 500, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)
    
    # Predictions.
    train_prediction = tf.nn.softmax(op)

(?, 10)
(100, 10, 27)


In [19]:
import os
num_steps = 4000
summary_frequency = 200


with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    saver = tf.train.Saver()
    prev_perplexity = None
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        global_step.assign_add(1)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            valid_batch = valid_batches.next()
            for i in range(num_unrollings):
                feed_dict[train_data[i]] = valid_batch[i]
            predictions = session.run([train_prediction], feed_dict=feed_dict)
            print(batches2string(valid_batch))
            labels = np.flip(valid_batch, axis = 0)
            print(batches2string(labels))
            predictions = np.reshape(predictions, [num_unrollings, batch_size, vocabulary_size])
            print(batches2string(predictions))
            perplexity = np.exp(logprob(predictions, labels))
            print('Minibatch perplexity: %.2f' % float(perplexity))
            print(os.getcwd())
            if prev_perplexity is None:
                prev_perplexity = perplexity
                print('saving %d' % step)
                saver.save(session, os.getcwd()+'/string_reversal_best')
            if perplexity < prev_perplexity:
                prev_perplexity = perplexity
                print('saving %d' % step)
                saver.save(session, os.getcwd()+'/string_reversal_best')

Initialized
Average loss at step 0: 3.331055 learning rate: 5.000000
['///////act', '//////that', '//////used', '///violent', '/////means', '////////to', '///destroy', '///////the', 'organizati', '////////of', '///society', '////////it', '///////has', '//////also', '//////been', '/////taken', '////////up', '////////as', '/////////a', '//positive', '/////label', '////////by', '//////self', '///defined', 'anarchists', '///////the', '//////word', '/anarchism', '////////is', '///derived', '//////from', '///////the', '/////greek', '///without', '///archons', '/////ruler', '/////chief', '//////king', '/anarchism', '////////as', '/////////a', '/political', 'philosophy', '////////is', '///////the', '////belief', '//////that', '////rulers', '///////are', 'unnecessar', '///////and', '////should', '////////be', '/abolished', '//although', '/////there', '///////are', '/differing', 'interpreta', '////////of', '//////what', '//////this', '/////means', '/anarchism', '//////also', '////refers', '/////

In [28]:
prediction = None
with tf.Session(graph = graph) as sess:
    saver = tf.train.import_meta_graph(os.getcwd()+'/string_reversal_best.meta')
    saver.restore(sess,tf.train.latest_checkpoint(os.getcwd()+'/'))
    cont = True
    while cont is True:
        st = input('Enter a string to reverse')
        st = st.split(' ')
        words_scaled = [(num_unrollings - len(word)) * '/' + word if len(word) < num_unrollings else word[:num_unrollings] for word in st]
        final_batch = list()
        for j in range(num_unrollings):
            batches = np.zeros([100, vocabulary_size])
            for i in range(len(words_scaled)):
                batches[i, char2id(words_scaled[i][j])] = 1
            final_batch.append(batches)
        final_batch = np.reshape(final_batch, [num_unrollings, 100, vocabulary_size])
        for i in range(num_unrollings):
            feed_dict[train_data[i]] = final_batch[i]
        prediction = sess.run([train_prediction], feed_dict=feed_dict)
        labels = np.flip(final_batch, axis = 0)
        print(batches2string(labels))
        prediction = np.reshape(prediction, [num_unrollings, 100, vocabulary_size])
        print(batches2string(prediction[:, 0:len(words_scaled), :]))
        words = batches2string(prediction[:, 0:len(words_scaled), :])
        fin_str = ''
        for word in words:
            while '/' in word:
                word = word.replace('/','')
            if fin_str is '':
                fin_str = word
            else:
                fin_str = fin_str + ' ' + word
        print(fin_str)
        ip = input('Do you want to continue? (Y/N)')
        if ip is 'N':
            cont = False

INFO:tensorflow:Restoring parameters from G:\PythonProjects\udacity_deep_learning/string_reversal_best
Enter a string to reversethe quick brown fox
['eht///////', 'kciuq/////', 'nworb/////', 'xof///////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////', '//////////